In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from functools import reduce

# Остановка существующего контекста (если есть)
try:
    sc.stop()
    print("Предыдущий SparkContext остановлен")
except:
    print("Нет активного SparkContext для остановки")

# Инициализация SparkContext
sc = SparkContext("local", "Longest Sequence")
print("SparkContext успешно инициализирован")

Нет активного SparkContext для остановки
SparkContext успешно инициализирован


In [2]:
# Создание RDD с числами для анализа
data = [3, 4, 5, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 15, 16, 17]
rdd = sc.parallelize(data)

# Получаем данные из RDD и работаем с ними локально
all_data = rdd.collect()
print("Исходные данные:", all_data)

Исходные данные: [3, 4, 5, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 15, 16, 17]


In [3]:
# Функция для нахождения последовательностей по возрастанию
def find_sequences_asc(numbers):
    sequences = []
    current_sequence = [numbers[0]]

    for i in range(1, len(numbers)):
        if numbers[i] == numbers[i - 1]+1:
            current_sequence.append(numbers[i])
        else:
            sequences.append(current_sequence)
            current_sequence = [numbers[i]]

    sequences.append(current_sequence)
    return sequences

# Функция для нахождения последовательностей по убыванию
def find_sequences_desc(numbers):
    sequences = []
    current_sequence = [numbers[0]]

    for i in range(1, len(numbers)):
        if numbers[i] == numbers[i - 1]-1:
            current_sequence.append(numbers[i])
        else:
            sequences.append(current_sequence)
            current_sequence = [numbers[i]]

    sequences.append(current_sequence)
    return sequences

In [4]:
# Имитируем поведение glom() и flatMap()
# Разбиваем данные на несколько частей для имитации партиций
partition_size = max(1, len(all_data) // 4)  # Предположим, 4 партиции
partitions = [all_data[i:i+partition_size] for i in range(0, len(all_data), partition_size)]
print("Имитация партиций:", partitions)

# Применяем функции к каждой "партиции"
sequences_asc = []
sequences_desc = []

for partition in partitions:
    # Сортируем данные внутри каждой партиции
    sorted_partition = sorted(partition)
    reversed_partition = sorted(partition, reverse=True)

    # Находим последовательности
    sequences_asc.extend(find_sequences_asc(sorted_partition))
    sequences_desc.extend(find_sequences_desc(reversed_partition))

# Фильтруем короткие последовательности (длиной 1)
sequences_asc = [seq for seq in sequences_asc if len(seq) > 1]
sequences_desc = [seq for seq in sequences_desc if len(seq) > 1]

print("Последовательности по возрастанию:", sequences_asc)
print("Последовательности по убыванию:", sequences_desc)

Имитация партиций: [[3, 4, 5, 1], [2, 3, 4, 5], [6, 8, 10, 11], [12, 15, 16, 17]]
Последовательности по возрастанию: [[3, 4, 5], [2, 3, 4, 5], [10, 11], [15, 16, 17]]
Последовательности по убыванию: [[5, 4, 3], [5, 4, 3, 2], [11, 10], [17, 16, 15]]


In [5]:
# Находим самые длинные последовательности
longest_sequence_asc = max(sequences_asc, key=len) if sequences_asc else []
longest_sequence_desc = max(sequences_desc, key=len) if sequences_desc else []

print("Самая длинная последовательность по возрастанию:", longest_sequence_asc)
print("Самая длинная последовательность по убыванию:", longest_sequence_desc)

# Выбираем более длинную последовательность
result = longest_sequence_asc if len(longest_sequence_asc) > len(longest_sequence_desc) else longest_sequence_desc

print(f"Самая длинная упорядоченная последовательность: {result}")

Самая длинная последовательность по возрастанию: [2, 3, 4, 5]
Самая длинная последовательность по убыванию: [5, 4, 3, 2]
Самая длинная упорядоченная последовательность: [5, 4, 3, 2]


In [6]:
# Останавливаем SparkContext
sc.stop()
print("SparkContext остановлен")

SparkContext остановлен
